[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/01_Code/02_Reacher_Env/02_Default_Environment/03_HF_DIRECT_RAD_REACHER/03_Model_testing_HF_D_Reacher_Reckless_Adaptive.ipynb)

In [ ]:
# Install the required libraries in your Google Colab environment
!pip install gymnasium mujoco stable-baselines3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 552.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 9.3 MB/s eta 0:00:00


In [ ]:
# Import the necessary libraries
import gymnasium as gym
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from google.colab import drive
from google.colab import data_table

# Mount Google Drive
drive.mount('/content/drive')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Mounted at /content/drive


Average Episodic Reward

In [ ]:
def average_episodic_reward(test_trajectory_df):
    # Step 1: Calculate Cumulative Reward Per Episode
    cumulative_rewards = test_trajectory_df.groupby('Episode')['Reward'].sum().reset_index()
    cumulative_rewards.columns = ['episode', 'cumulative_reward']

    # Step 2: Calculate Variance and Standard Deviation of Cumulative Rewards
    variance = cumulative_rewards['cumulative_reward'].var()
    std_dev = cumulative_rewards['cumulative_reward'].std()
    final_reward = cumulative_rewards['cumulative_reward'].mean()

    # Step 4: Return the dataframe containing all results
    return cumulative_rewards, variance, std_dev, final_reward

COMBINING ALL THE DATA FRAMES

In [ ]:
def create_cummulative_reward(df1, df2, df3, df4, df5):
    """
    Create a data frame 'ideal_cummulative_reward' that contains:
      - 'episode' column (common across all data frames)
      - 'cumulative_reward_1' to 'cumulative_reward_5' columns from each data frame respectively,
        where each input data frame has columns 'episode' and 'cummulative_reward'.
      - 'mean_cumulative_reward' column containing the row-wise mean of the 5 cumulative rewards.

    Parameters:
      df1, df2, df3, df4, df5 (pd.DataFrame): Data frames with columns 'episode' and 'cummulative_reward'.

    Returns:
      pd.DataFrame: The merged and aggregated data frame.
    """
    # Rename the 'cummulative_reward' column in each data frame to a unique name.
    df1_renamed = df1.rename(columns={'cumulative_reward': 'cumulative_reward_1'})
    df2_renamed = df2.rename(columns={'cumulative_reward': 'cumulative_reward_2'})
    df3_renamed = df3.rename(columns={'cumulative_reward': 'cumulative_reward_3'})
    df4_renamed = df4.rename(columns={'cumulative_reward': 'cumulative_reward_4'})
    df5_renamed = df5.rename(columns={'cumulative_reward': 'cumulative_reward_5'})

    # Merge the data frames on the 'episode' column.
    merged_df = df1_renamed[['episode', 'cumulative_reward_1']].copy()
    merged_df = merged_df.merge(df2_renamed[['episode', 'cumulative_reward_2']], on='episode')
    merged_df = merged_df.merge(df3_renamed[['episode', 'cumulative_reward_3']], on='episode')
    merged_df = merged_df.merge(df4_renamed[['episode', 'cumulative_reward_4']], on='episode')
    merged_df = merged_df.merge(df5_renamed[['episode', 'cumulative_reward_5']], on='episode')

    # Compute the episode-wise mean of the cumulative rewards.
    reward_columns = [
        'cumulative_reward_1',
        'cumulative_reward_2',
        'cumulative_reward_3',
        'cumulative_reward_4',
        'cumulative_reward_5'
    ]
    merged_df['mean_cumulative_reward'] = merged_df[reward_columns].mean(axis=1)

    return merged_df

VARIANCE, STANDARD DEVIATION & FINAL REWARD

In [ ]:
def calculate_performance(variance_values, std_dev_values, final_reward_values):
    """
    Calculate performance metrics by computing the mean of each provided list.

    Parameters:
        variance_values (list of float): A list containing 5 variance values.
        std_dev_values (list of float): A list containing 5 standard deviation values.
        final_reward_values (list of float): A list containing 5 final reward values.

    Returns:
        tuple: A tuple containing three values:
            - mean_variance (float): Mean of the variance values.
            - mean_std_dev (float): Mean of the standard deviation values.
            - mean_final_reward (float): Mean of the final reward values.
    """
    mean_variance = sum(variance_values) / len(variance_values)
    mean_std_dev = sum(std_dev_values) / len(std_dev_values)
    mean_final_reward = sum(final_reward_values) / len(final_reward_values)


    return mean_variance, mean_std_dev, mean_final_reward


0. MODEL TESTING (HF_D_Ideal)

In [ ]:
trajectory_df_1 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/3_hf_d_reckless/1_hf_d_reacher_reckless_df_3.pkl')      # Update directory location 1
trajectory_df_2 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/3_hf_d_reckless/2_hf_d_reacher_reckless_df_3.pkl')      # Update directory location 2
trajectory_df_3 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/3_hf_d_reckless/3_hf_d_reacher_reckless_df_3.pkl')      # Update directory location 3
trajectory_df_4 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/3_hf_d_reckless/4_hf_d_reacher_reckless_df_3.pkl')      # Update directory location 4
trajectory_df_5 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/03_test_trajectories/3_hf_d_reckless/5_hf_d_reacher_reckless_df_3.pkl')      # Update directory location 5

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


TESTING THE MODEL BY ANALYZING THE DATA FRAME <br>



1. Human feedback Direct Reacher Reckless Adaptive data frame 1<br>
trajectory_df_1 ---> 1_hf_d_reacher_conservative_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_1)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_1)

In [ ]:
learning_curve_result_df_1, variance_1, std_dev_1, average_episodic_reward_1 = average_episodic_reward(trajectory_df_1)
print("Variance:", variance_1)
print("Standard Deviation:", std_dev_1)
print("Average Episodic Reward:", average_episodic_reward_1)

Variance: 5.099706862034824e-29
Standard Deviation: 7.141223187966347e-15
Average Episodic Reward: -30.3363030103983


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_1)

,episode,cumulative_reward
0,1,-30.336303
1,2,-30.336303
2,3,-30.336303
3,4,-30.336303
4,5,-30.336303
...,...,...
95,96,-30.336303
96,97,-30.336303
97,98,-30.336303
98,99,-30.336303


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


2. Human feedback Direct Reacher Reckless Adaptive data frame 2<br>
trajectory_df_2 ---> 2_hf_d_reacher_conservative_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_2)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_2)

In [ ]:
learning_curve_result_df_2, variance_2, std_dev_2, average_episodic_reward_2 = average_episodic_reward(trajectory_df_2)
print("Variance:", variance_2)
print("Standard Deviation:", std_dev_2)
print("Average Episodic Reward:", average_episodic_reward_2)

Variance: 6.247140905992659e-28
Standard Deviation: 2.4994281157882216e-14
Average Episodic Reward: -30.250896691674516


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_2)

,episode,cumulative_reward
0,1,-30.250897
1,2,-30.250897
2,3,-30.250897
3,4,-30.250897
4,5,-30.250897
...,...,...
95,96,-30.250897
96,97,-30.250897
97,98,-30.250897
98,99,-30.250897


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


3. Human feedback Direct Reacher Conservative data frame 3<br>
trajectory_df_3 ---> 3_hf_d_reacher_conservative_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_3)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_3)

In [ ]:
learning_curve_result_df_3, variance_3, std_dev_3, average_episodic_reward_3 = average_episodic_reward(trajectory_df_3)
print("Variance:", variance_3)
print("Standard Deviation:", std_dev_3)
print("Average Episodic Reward:", average_episodic_reward_3)

Variance: 1.274926715508706e-27
Standard Deviation: 3.5706115939831735e-14
Average Episodic Reward: -27.127388551150084


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_3)

,episode,cumulative_reward
0,1,-27.127389
1,2,-27.127389
2,3,-27.127389
3,4,-27.127389
4,5,-27.127389
...,...,...
95,96,-27.127389
96,97,-27.127389
97,98,-27.127389
98,99,-27.127389


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


4. Human feedback Direct Reacher Reckless Adaptive data frame 4<br>
trajectory_df_4 ---> 4_hf_d_reacher_conservative_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_4)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_4)

In [ ]:
learning_curve_result_df_4, variance_4, std_dev_4, average_episodic_reward_4 = average_episodic_reward(trajectory_df_4)
print("Variance:", variance_4)
print("Standard Deviation:", std_dev_4)
print("Average Episodic Reward:", average_episodic_reward_4)

Variance: 3.2638123917022874e-27
Standard Deviation: 5.712978550373078e-14
Average Episodic Reward: -32.448851345396086


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_4)

,episode,cumulative_reward
0,1,-32.448851
1,2,-32.448851
2,3,-32.448851
3,4,-32.448851
4,5,-32.448851
...,...,...
95,96,-32.448851
96,97,-32.448851
97,98,-32.448851
98,99,-32.448851


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


5. Human feedback Direct Reacher Reckless Adaptive data frame 5<br>
trajectory_df_5 ---> 5_hf_d_reacher_conservative_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_5)

Output hidden; open in https://colab.research.google.com to view.

A. Average Episodic Reward (trajectory_df_5)

In [ ]:
learning_curve_result_df_5, variance_5, std_dev_5, average_episodic_reward_5 = average_episodic_reward(trajectory_df_5)
print("Variance:", variance_5)
print("Standard Deviation:", std_dev_5)
print("Average Episodic Reward:", average_episodic_reward_5)

Variance: 1.0326906395620518e-27
Standard Deviation: 3.213550434584856e-14
Average Episodic Reward: -29.882489685858168


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_5)

,episode,cumulative_reward
0,1,-29.88249
1,2,-29.88249
2,3,-29.88249
3,4,-29.88249
4,5,-29.88249
...,...,...
95,96,-29.88249
96,97,-29.88249
97,98,-29.88249
98,99,-29.88249


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


COMBINED DATA FRAMES - HF-D-Reckless Adaptive

In [ ]:
cummulative_reward = create_cummulative_reward(
        learning_curve_result_df_1,
        learning_curve_result_df_2,
        learning_curve_result_df_3,
        learning_curve_result_df_4,
        learning_curve_result_df_5
    )

data_table.enable_dataframe_formatter()
data_table.DataTable(cummulative_reward)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,episode,cumulative_reward_1,cumulative_reward_2,cumulative_reward_3,cumulative_reward_4,cumulative_reward_5,mean_cumulative_reward
0,1,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
1,2,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
2,3,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
3,4,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
4,5,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
...,...,...,...,...,...,...,...
95,96,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
96,97,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
97,98,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186
98,99,-30.336303,-30.250897,-27.127389,-32.448851,-29.88249,-30.009186


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


VARIANCE, STANDARD DEVIATION & FINAL REWARD : HF-D-Reckless Adaptive

In [ ]:
 # Organize the values into lists
variance_list = [variance_1, variance_2, variance_3, variance_4, variance_5]
std_dev_list = [std_dev_1, std_dev_2, std_dev_3, std_dev_4, std_dev_5]
final_reward_list = [average_episodic_reward_1, average_episodic_reward_2, average_episodic_reward_3, average_episodic_reward_4, average_episodic_reward_5]

# Call the function to compute the mean performance values
mean_variance, mean_std_dev, mean_final_reward  = calculate_performance(variance_list, std_dev_list, final_reward_list)

# Print the results
print("HF_D_RAD Mean Variance:", mean_variance)
print("HF_D_RAD  Mean Standard Deviation:", mean_std_dev)
print("HF_D_RAD  Mean Average Episodic Reward:", mean_final_reward)


HF_D_RAD Mean Variance: 1.2494281811985318e-27
HF_D_RAD  Mean Standard Deviation: 3.142138202705193e-14
HF_D_RAD  Mean Average Episodic Reward: -30.00918585689543
